In [25]:
import cv2 as cv

In [26]:
# Göreceli yol ile dosyalara erişim

#path      = "../00_external_models/GOOGLENET/models/Face_Detection"

model_bin   = "../00_external_models/GOOGLENET/models/Face_Detection/res10_300x300_ssd_iter_140000_fp16.caffemodel"
config_text = "../00_external_models/GOOGLENET/models/Face_Detection/deploy.prototxt"

In [27]:
net = cv.dnn.readNetFromCaffe(config_text, model_bin)

In [28]:
# set back-end
net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)

In [29]:
cap = cv.VideoCapture("friend_group.mp4")

In [30]:
while cap.isOpened():
    ret, image = cap.read()

    if not ret: # Eğer kare alınamadıysa döngüden çık
        print("Video sonuna ulaşıldı veya dosya okunamıyor.")
        break

    image = cv.flip(image, 1)
    image = cv.resize(image, (600, 400))
    h, w = image.shape[:2]
    
    blobImage = cv.dnn.blobFromImage(image,
                                     1.0,
                                     (300, 300),
                                     (104.0, 177.0, 123.0),
                                     False,
                                     False)
    net.setInput(blobImage)
    cvOut = net.forward()

    t, _ = net.getPerfProfile()
    fps = 1000 / (t * 1000.0 / cv.getTickFrequency())
    label = 'FPS: %.2f' % fps
    cv.putText(image,
               label,
               (0, 15),
               cv.FONT_HERSHEY_SIMPLEX,
               0.5,
               (0, 255, 0))
    
    for detection in cvOut[0, 0, :, :]:
        score = float(detection[2])
        objIndex = int(detection[1])
        if score > 0.5:
            left   = detection[3] * w
            top    = detection[4] * h
            right  = detection[5] * w
            bottom = detection[6] * h
            cv.rectangle(image,
                         (int(left), int(top)),
                         (int(right), int(bottom)),
                         (255, 0, 0),
                         thickness=2)
            cv.putText(image,
                       "score:%.6f" % score,
                       (int(left), int(top)),
                       cv.FONT_HERSHEY_SIMPLEX,
                       0.5,
                       (0, 0, 255),
                       1)
    
    cv.imshow('face-detection-demo', image)
    c = cv.waitKey(10)
    if c == 27:
        break
cap.release()
cv.destroyAllWindows()

Video sonuna ulaşıldı veya dosya okunamıyor.


![](result1.png)